# Makeover Monday | 2026 W4 Job titles in Highest Deman - State by State

The data for this weeks Makeover Monday challenge came from the article published by Lightcast "[Job titles in highest-demand, state by state](https://lightcast.io/resources/blog/most-posted-for-jobs-in-each-us-state)" (published on Nov 19, 2025 by Hannah Grieser & JP Lespinasse). The article provides a high-level view of the overall national and state-level hiring trends and includes an interactive US map encouraging users to explore the data for themselves. The data used in this notebook and the final Tableau Dashboard was prepared by the [Makeover Monday](https://makeovermonday.co.uk/), for use in the Tableau Makeover Monday Data Visualization challenge.

**How Lightcast Collects their Data**

I was interested in understanding how this data was collected, so I took a closer look at Lightcast’s methodology. Lightcast is a labor market and economic data intelligence company that specializes in large-scale job posting and workforce data across the world.
Their data is compiled from millions of online job postings, employer career sites, and job boards, which are then cleaned and refined by AI and human experts to create a comprehensive database of job posting activity.

see their [About : How We Do It](https://lightcast.io/why-lightcast/about) section on their website for mor information



* **Article Link**: https://lightcast.io/resources/blog/most-posted-for-jobs-in-each-us-state

* **Compiled Data**: (*publisehd by Makeover Monday*):https://data.world/makeovermonday/2025w4-most-posted-us-jobs-by-state


In [49]:
import pandas as pd
import pandas_gbq
import numpy as np
import geopandas as gpd #you need this to use the spatial files

In [ ]:
df = pd.read_csv('https://query.data.world/s/cexlh3qmmucu3oifgigaual53fahxm?dws=00000',sep=';')

## Exploratory Data Analysis

In [ ]:
df.describe(include='all').T

,count,unique,top,freq
State Name,50,50,Alabama,1
1st Place,50,2,Registered Nurse,49
2nd Place,50,5,Retail Sales Associate,31
3rd Place,50,8,Tractor-Trailer Truck Driver,16


In [ ]:
df.head()

,State Name,1st Place,2nd Place,3rd Place
0,Alabama,Registered Nurse,Tractor-Trailer Truck Driver,Retail Sales Associate
1,Alaska,Registered Nurse,Retail Sales Associate,Physician
2,Arizona,Registered Nurse,Retail Sales Associate,Physician
3,Arkansas,Registered Nurse,Tractor-Trailer Truck Driver,Retail Sales Associate
4,California,Registered Nurse,Retail Sales Associate,Sales Representative


In [ ]:
first_place = df['1st Place'].unique()
print(first_place)

['Registered Nurse' 'Retail Sales Associate']


In [ ]:
second_place = df['2nd Place'].unique()
print(second_place)

['Tractor-Trailer Truck Driver' 'Retail Sales Associate'
 'Software Developer / Engineer' 'Physician' 'Registered Nurse']


In [ ]:
third_place = df['3rd Place'].unique()
print(third_place)

['Retail Sales Associate' 'Physician' 'Sales Representative'
 'Tractor-Trailer Truck Driver' 'Customer Service Representative'
 'Retail Store Manager / Supervisor' 'Software Developer / Engineer'
 'Licensed Practical / Vocational Nurse']


# Plan for Tableau Viz: **Top Jobs vs Popular Jobs**

The plan for this visualization is to explore the relationship between top-ranked jobs (1st place by state) and overall job popularity across the US using a simple bar chart paired with a supporting hexmap.

**Have you ever heard of Ranked Choice Voting?**

This was the first thing that came to mind when looking at this data, and I immediately thought - how can I show the interaction between the top job (1st place - ranked choice) and the most popular jobs in the USA. Maybe they are the same, maybe they aren't.
There isn't a lot of data processing that needs to be done for this, the data is all here we just need to reshape it to be "long" so my "melting technique" which I use for everything will apply - and then we just count!

**Hexmap to show Highlight Geospatial Relationship**

My favorite way to show US state data (when you dont really care about where in the state the data is coming from) is to use a Hexmap and I find myself going back to this saved YouTube video over and over :
[Tableau Hexagon Map Tutorial: How to Create Padded & Non-Padded Hex Maps](https://www.youtube.com/watch?v=f4teeqBkwLs).
The Hexmap in my viz is not going to do the heavy lifting, its going to be an aside in the dashboard - what I really want to show is the bar chart. I will be enriching my df_long with the spatial files from https://stanke.co/hexstatespadded/

## Data Processing

In [ ]:
#clean up the column names to snake_case
df_1 = df.copy()
df_1.columns = (
    df.columns
      .astype(str)
      .str.strip()
      .str.replace(r"\s+", "_", regex=True)
      .str.lower()
)

df_1.head()

,state_name,1st_place,2nd_place,3rd_place
0,Alabama,Registered Nurse,Tractor-Trailer Truck Driver,Retail Sales Associate
1,Alaska,Registered Nurse,Retail Sales Associate,Physician
2,Arizona,Registered Nurse,Retail Sales Associate,Physician
3,Arkansas,Registered Nurse,Tractor-Trailer Truck Driver,Retail Sales Associate
4,California,Registered Nurse,Retail Sales Associate,Sales Representative


In [ ]:
##create the df_long which unpivot the data

df_1['count'] = 1

df_long = (
    df_1
      .melt(
          id_vars=["state_name"],
          var_name='rank_text',
          value_name='job_title'
      )
)

df_long['rank'] = df_long['rank_text'].str.replace(r'st_place|nd_place|rd_place', '', regex=True) #This replaces '1st_place' with '1', '2nd_place' with '2', etc.
df_long

,state_name,rank_text,job_title,rank
0,Alabama,1st_place,Registered Nurse,1
1,Alaska,1st_place,Registered Nurse,1
2,Arizona,1st_place,Registered Nurse,1
3,Arkansas,1st_place,Registered Nurse,1
4,California,1st_place,Registered Nurse,1
...,...,...,...,...
195,Virginia,count,1,count
196,Washington,count,1,count
197,West Virginia,count,1,count
198,Wisconsin,count,1,count


In [ ]:
job_title_count = df_long['job_title'].nunique()
job_title = df_long['job_title'].unique()
print(f'{job_title_count} unique jobs'),
print(job_title)

10 unique jobs
['Registered Nurse' 'Retail Sales Associate'
 'Tractor-Trailer Truck Driver' 'Software Developer / Engineer'
 'Physician' 'Sales Representative' 'Customer Service Representative'
 'Retail Store Manager / Supervisor'
 'Licensed Practical / Vocational Nurse' 1]


### Enrich the Data with the Spatial Data

In [61]:
from google.colab import drive
drive.mount('/content/drive')

NotImplementedError: google.colab.drive.mount is not supported in Colab Enterprise.

In [51]:
hex_gdf = gpd.read_file("HexStatesPadded/HexStates.shp")

DataSourceError: HexStatesPadded/HexStates.shp: No such file or directory